<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Business Analytics e Machine Learning Para Projetos de Data Science</font>
## <font color='blue'>Projeto 8</font>
### <font color='blue'>Identificando e Prevendo os Fatores Que Contribuem Para a Satisfação dos Funcionários</font>

## Instalando e Carregando os Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
# Imports
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando os Dados

In [4]:
# Carrega o dataset
df_dsa = pd.read_csv('dataset.csv')

In [5]:
# Shape
df_dsa.shape

(23058, 30)

In [6]:
# Amostra
df_dsa.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Employee Source,AgeStartedWorking
0,41,Voluntary Resignation,Travel_Rarely,Sales,1,2,Life Sciences,2,Female,3,...,0,8,0,1,6,4,0,5,Referral,33
1,37,Voluntary Resignation,Travel_Rarely,Human Resources,6,4,Human Resources,1,Female,3,...,0,8,0,1,6,4,0,5,Referral,29
2,41,Voluntary Resignation,Travel_Rarely,Sales,1,2,Life Sciences,2,Female,3,...,0,8,0,1,6,4,0,5,Referral,33
3,37,Voluntary Resignation,Travel_Rarely,Human Resources,6,4,Marketing,1,Female,3,...,0,8,0,1,6,4,0,5,Referral,29
4,37,Voluntary Resignation,Travel_Rarely,Human Resources,6,4,Human Resources,1,Female,3,...,0,8,0,1,6,4,0,5,Referral,29


In [7]:
# Info
df_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23058 entries, 0 to 23057
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       23058 non-null  int64 
 1   Attrition                 23058 non-null  object
 2   BusinessTravel            23058 non-null  object
 3   Department                23058 non-null  object
 4   DistanceFromHome          23058 non-null  int64 
 5   Education                 23058 non-null  int64 
 6   EducationField            23058 non-null  object
 7   EnvironmentSatisfaction   23058 non-null  int64 
 8   Gender                    23058 non-null  object
 9   JobInvolvement            23058 non-null  int64 
 10  JobLevel                  23058 non-null  int64 
 11  JobRole                   23058 non-null  object
 12  JobSatisfaction           23058 non-null  int64 
 13  MaritalStatus             23058 non-null  object
 14  MonthlyIncome         

Veja o dicionário de dados no Capítulo 16 do curso.

## Seleção da Variável Alvo

In [8]:
df_dsa['Attrition'].value_counts()

Attrition
Current employee         19370
Voluntary Resignation     3601
Termination                 87
Name: count, dtype: int64

## Encode da Variável Alvo e Preparação dos Dados

Vamos converter o problema em aprendizado supervisionado, com a variável alvo sendo binária. Classes:

- Classe 0 - Não ocorrência do evento (não pediu demissão).
- Classe 1 - Ocorrência do evento (pediu demissão).

Vamos analisar os resultados com base na classe 1 e compreender os fatores que influenciam a satisfação dos funcionários, ou seja, levam os funcionários a pedir demissão.

A categoria Termination será descartada, pois a decisão de demissão nesse caso foi da empresa e não do funcionário!

In [9]:
# Filtrar o dataframe para manter apenas 'Current employee' e 'Voluntary Resignation'
df_dsa = df_dsa[df_dsa['Attrition'].isin(['Current employee', 'Voluntary Resignation'])]

In [10]:
# Verificar os valores únicos da coluna 'Attrition' após o filtro
df_dsa['Attrition'].value_counts()

Attrition
Current employee         19370
Voluntary Resignation     3601
Name: count, dtype: int64

In [11]:
# Encode da variável alvo
df_dsa['Attrition'] = df_dsa['Attrition'].apply(lambda x: 1 if x == 'Voluntary Resignation' else 0)

In [12]:
df_dsa['Attrition'].value_counts()

Attrition
0    19370
1     3601
Name: count, dtype: int64

In [13]:
# Separação das variáveis
X = df_dsa.drop('Attrition', axis = 1)
y = df_dsa['Attrition']

In [14]:
# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
# Separação das variáveis numéricas e categóricas
cat_features = X.select_dtypes(include = ['object']).columns.tolist()
num_features = X.select_dtypes(include = ['int64', 'float64']).columns.tolist()

## Pipeline de Pré-Processamento de Variáveis Numéricas

O objetivo deste pipeline é garantir que todas as features numéricas no conjunto de dados sejam tratadas de forma consistente e apropriada antes de serem alimentadas no modelo de Machine Learning.

- Tratamento de valores ausentes: Substitui valores ausentes pela mediana para evitar que o modelo seja afetado por dados faltantes.
- Normalização: Padroniza os dados para que todas as features numéricas tenham a mesma escala, melhorando a performance do modelo e garantindo que nenhuma feature domine as outras devido à escala.

In [16]:
# Cria o pipeline
numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())])

## Pipeline de Pré-Processamento de Variáveis Categóricas

O objetivo deste pipeline é garantir que todas as features categóricas no conjunto de dados sejam tratadas de forma consistente e apropriada antes de serem alimentadas no modelo de Machine Learning.

- Tratamento de valores ausentes: Substitui valores ausentes por 'missing', criando uma categoria especial para valores ausentes.
- Codificação One-Hot: Transforma as features categóricas em uma forma que pode ser usada pelo modelo de Machine Learning, convertendo cada categoria em uma coluna binária.

In [17]:
# Cria o pipeline
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

## Pipeline de Modelagem

A classe ColumnTransformer permite aplicar diferentes transformações a diferentes subsets de features. Isso é útil quando você precisa pré-processar colunas numéricas e categóricas de maneira distinta.

In [18]:
# Combinando os passos de pré-processamento
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)])

O objetivo do pipeline de modelagem é combinar todas as etapas de pré-processamento e modelagem em um único fluxo de trabalho que pode ser aplicado de forma consistente aos dados de treinamento e de teste. Isso garante que todas as transformações necessárias sejam aplicadas corretamente e na ordem certa, facilitando a replicação e manutenção do processo.

In [19]:
# Criando o pipeline de modelagem
modelo_dsa = Pipeline(steps = [('preprocessor', preprocessor), 
                               ('classifier', LogisticRegression(max_iter = 1000))])

In [20]:
# Treina o modelo
modelo_dsa.fit(X_treino, y_treino)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'DistanceFromHome',
                                                   'Education',
                                                   'EnvironmentSatisfaction',
                                                   'JobInvolvement', 'JobLevel',
                                                   'JobSatisfaction',
                                                   'MonthlyIncome',
                                                   'NumCompaniesWorked',
                                                   'PercentSalaryHike',
                                                   'PerformanceRating',
                                                   '...
                                                   'YearsWithCurrManager',
                                                   'AgeStartedWorking ']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BusinessTravel',
                                                   'Department',
                                                   'EducationField', 'Gender',
                                                   'JobRole', 'MaritalStatus',
                                                   'OverTime',
                                                   'Employee Source'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [21]:
# Previsões com os dados de teste
y_pred = modelo_dsa.predict(X_teste)

In [22]:
# Avalia o modelo
acuracia = accuracy_score(y_teste, y_pred)

In [23]:
print(acuracia)

0.8550598476605006


## Analisando os Coeficientes do Modelo

In [24]:
# Analisando os coeficientes do modelo
coefficients = modelo_dsa.named_steps['classifier'].coef_[0]

In [25]:
# Nomes dos atributos
feature_names = num_features + list(modelo_dsa.named_steps['preprocessor'] \
                                    .transformers_[1][1].named_steps['onehot'].get_feature_names_out(cat_features))

In [26]:
feature_names

['Age',
 'DistanceFromHome',
 'Education',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager',
 'AgeStartedWorking ',
 'BusinessTravel_Non-Travel',
 'BusinessTravel_Travel_Frequently',
 'BusinessTravel_Travel_Rarely',
 'Department_Human Resources',
 'Department_Research & Development',
 'Department_Sales',
 'EducationField_Human Resources',
 'EducationField_Life Sciences',
 'EducationField_Marketing',
 'EducationField_Medical',
 'EducationField_Other',
 'EducationField_Technical Degree',
 'Gender_Female',
 'Gender_Male',
 'JobRole_Healthcare Representative',
 'JobRole_Human Resources',
 'JobRole_Laboratory Technician',
 'JobRole_Manager',
 'JobRole_Manufacturing Dir

In [27]:
# Dataframe
coeff_df = pd.DataFrame({'Atributo': feature_names, 'Coeficiente': coefficients}).sort_values(by = 'Coeficiente', ascending = False)

In [28]:
# Display 
coeff_df.head(10)

,Atributo,Coeficiente
22,BusinessTravel_Travel_Frequently,0.494839
32,EducationField_Technical Degree,0.275768
56,Employee Source_Referral,0.257281
46,MaritalStatus_Single,0.213240
37,JobRole_Laboratory Technician,0.213197
48,OverTime_Yes,0.183383
1,DistanceFromHome,0.160642
18,YearsSinceLastPromotion,0.157113
43,JobRole_Sales Representative,0.126106
16,YearsAtCompany,0.105078


Os coeficientes na regressão logística indicam a força e a direção da associação entre cada feature (atributo) e a probabilidade de ocorrência do evento alvo, que neste caso é a demissão voluntária (Attrition). 
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

Ou seja, a análise dos coeficientes de um modelo de regressão logística nos ajuda a entender a influência de cada atributo na probabilidade do evento de interesse. Os coeficientes positivos indicam que, conforme o valor do atributo aumenta, a probabilidade do funcionário se demitir voluntariamente também aumenta. 

Vamos interpretar os 10 coeficientes com maior valor:

**BusinessTravel_Travel_Frequently (0.494839)**

Funcionários que viajam frequentemente a negócios têm uma maior probabilidade de se demitir voluntariamente. Esse coeficiente é bastante significativo, sugerindo que a frequência de viagens pode ser um fator de estresse ou insatisfação.

**EducationField_Technical Degree (0.275768)**

Funcionários com um diploma técnico têm uma probabilidade maior de se demitir voluntariamente em comparação com aqueles de outras áreas educacionais. Isso pode indicar que esses funcionários têm mais oportunidades no mercado de trabalho ou que suas expectativas não estão sendo atendidas.

**Employee Source_Referral (0.257281)**

Funcionários que foram contratados por meio de indicações (Referral) têm uma probabilidade maior de se demitir voluntariamente. Isso pode sugerir que, apesar de serem indicados, eles podem não estar tão alinhados com a empresa quanto outros funcionários.

**MaritalStatus_Single (0.213240)**

Funcionários solteiros têm uma maior probabilidade de se demitir voluntariamente em comparação com funcionários casados ou em outros estados civis. Isso pode ser devido à maior flexibilidade e menos responsabilidades pessoais.

**JobRole_Laboratory Technician (0.213197)**

Funcionários que trabalham como técnicos de laboratório têm uma maior probabilidade de se demitir voluntariamente. Isso pode indicar insatisfação com a função específica ou o ambiente de trabalho.

**OverTime_Yes (0.183383)**

Funcionários que fazem horas extras têm uma maior probabilidade de se demitir voluntariamente. Isso sugere que o excesso de trabalho pode levar ao desgaste e à insatisfação.

**DistanceFromHome (0.160642)**

Maior distância de casa para o trabalho está associada a uma maior probabilidade de demissão voluntária. Longos deslocamentos podem causar cansaço e insatisfação.

**YearsSinceLastPromotion (0.157113)**

Funcionários que passaram mais anos desde a última promoção têm uma maior probabilidade de se demitir voluntariamente. Isso pode indicar insatisfação com as oportunidades de crescimento na empresa.

**JobRole_Sales Representative (0.126106)**

Funcionários que trabalham como representantes de vendas têm uma maior probabilidade de se demitir voluntariamente. Essa função pode ter alta pressão de desempenho ou falta de suporte adequado.

**YearsAtCompany (0.105078)**

Quanto mais anos um funcionário passa na empresa, maior é a probabilidade de se demitir voluntariamente. Isso pode indicar que, após um certo período, os funcionários podem sentir estagnação ou buscar novas oportunidades.

**Conclusão:**

Os coeficientes positivos indicam que esses fatores aumentam a probabilidade de demissão voluntária. Entender esses fatores pode ajudar a empresa a tomar medidas preventivas, como melhorar as condições de trabalho, oferecer oportunidades de crescimento e minimizar a necessidade de horas extras, para reduzir a taxa de rotatividade voluntária.

In [29]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [30]:
#%watermark -v -m

In [31]:
#%watermark --iversions

# Fim